In [39]:
#!pip install tensorflow


In [40]:
import db1
import ApoKomb1    
import torch 
from torch_geometric.nn import GCNConv
import torch.nn as nn
from torch_geometric.data import Data
import pandas as pd
import numpy as np


citys = db1.basi()

citys1 = citys.sample(50)
lats_c = citys1[["lat"]]
lngs_c = citys1[["lng"]]
node_citys =  np.array(citys1["city"])

location = citys1[["lat", "lng"]].astype(float)
#edw xrisimopoioume ena DataFrame gia na mporoume na exoume to ka8e edge se sigkekrimeno ar8imo gia na
#mhn xa8oume sthn sunexia apo pio polu paei se pia 


unique_node_citys= citys1["city"]
unique_node_citys = pd.DataFrame(data={
    'city' : unique_node_citys,
    'mappedId': pd.RangeIndex(len(unique_node_citys))
})



coords = ApoKomb1.sindet(lats_c, lngs_c)
edge_index = ApoKomb1.edge_index(node_citys)
distances = ApoKomb1.apostasis(node_citys, coords)

print(node_citys)
print(coords)
#print(edge_index)
#print(distances)

['Schimatári' 'Firá' 'Mantoúdi' 'Lixoúri' 'Perivóli' 'Tzermiádo' 'Níkaia'
 'Mýkonos' 'Skýros' 'Áno Kalentíni' 'Néa Výssa' 'Voúla' 'Kallithéa'
 'Smínthi' 'Kremastí' 'Ierissós' 'Eleónas' 'Préveza' 'Alistráti'
 'Igoumenítsa' 'Ampelákia' 'Síkinos' 'Kos' 'Argalastí' 'Galátista'
 'Ligourió' 'Míthymna' 'Pýrgos' 'Pérama' 'Chiliomódi' 'Kontokáli' 'Mália'
 'Kýthnos' 'Palaiokómi' 'Anatolikó' 'Loutráki' 'Skýdra' 'Antimácheia'
 'Amfilochía' 'Antíparos' 'Megísti' 'Xylókastro' 'Makrakómi' 'Isthmía'
 'Thessaloníki' 'Kímolos' 'Chaïdári' 'Áno Sýros' 'Káto Goúves' 'Orestiáda']
[[38.35   23.5833]
 [36.4102 25.4056]
 [38.7981 23.4797]
 [38.2    20.4333]
 [39.4208 20.0139]
 [35.1983 25.49  ]
 [39.5667 22.4667]
 [37.4453 25.3287]
 [38.9064 24.5658]
 [39.2515 21.1679]
 [41.5833 26.5333]
 [37.8459 23.7626]
 [40.275  22.575 ]
 [41.2289 24.8804]
 [36.4105 28.1191]
 [40.3983 23.8783]
 [38.3633 23.4428]
 [38.95   20.75  ]
 [41.0633 23.9581]
 [39.5034 20.2673]
 [37.95   23.5167]
 [36.6833 25.1167]
 [36.8153 27.1103

In [41]:
import networkx as nx
import numpy as mp
#from haversine import haversine

#sto sigkekrimeno meros tou kwdika pernoume ta dedomena apo prin kai ta topo8etoume se enan algorithmo gia na 
#broume tis apostaseis apo ka8e kombo, sth synexia ta bazoume ta apotelesmata os (EDGE FEATURES) kai meta
#Ftiaxnoume ena grafima me auta kai sth sunexia to metatrepoume se gramiko grafima gia na exoume ta (EDGE FEATURES) os (NODE FEATURES)

# Example usage:
# Create a sample graph
G = nx.Graph()
for i in range(len(edge_index)):
    G.add_edge(edge_index[i][0], edge_index[i][1], weight=distances[i])

def line_graph(G):
    L = nx.line_graph(G)
    for node in L.nodes:
        edges = G.edges(node)
        if edges:
            edge_features = G.edges[node]['weight']
            L.nodes[node]['e_f'] = edge_features
        else:
            L.nodes[node]['e_f'] = None
    return L

def line_graph_node_features(G):
    node_features = []
    for _, node_data in G.nodes(data=True):
        node_features.append(node_data['e_f'])
    return node_features

L = line_graph(G)
node_features1 = line_graph_node_features(L)

# Create a mapping from original graph edges to line graph nodes
edge_to_index = {edge: idx for idx, edge in enumerate(L.nodes())}

# Prepare edge index for the line graph
edge_index_t = []
for edge in L.edges:
    src, dst = edge
    edge_index_t.append([edge_to_index[src], edge_to_index[dst]])

# Convert edge list to a 2D tensor
edge_index_lg = torch.tensor(edge_index_t, dtype=torch.long).t().contiguous()
print(edge_index_lg)
# Convert node features to tensor
#nf = torch.tensor(node_features1, dtype=torch.float32).unsqueeze(1)


#nx.draw(G, with_labels= True)
# Transform the graph into its line graph representation
print('Original Graph: ',G)


#nx.draw(L, with_labels= True)
print('Line Graph: ',L)






tensor([[   0,    0,    0,  ..., 1218, 1219, 1220],
        [   1,  652,  379,  ..., 1224, 1223, 1221]])
Original Graph:  Graph with 50 nodes and 1225 edges
Line Graph:  Graph with 1225 nodes and 58800 edges


In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx

#edw dimiourgiete to nevroniko diktio to opoio exei eksis kommatia 
#1) To EmbeddingLayer to opoio exei os apotelesma sto forward (h0ij = Wxij + b,)
#2) Ena Graph Attection Layer ftMHA(htij , L(G))
#3) Ena Feed Forward Layer  ftFF(h˙t+1ij)
#4) Ena Output Layer to opoio meta apo epeksergasia apo ola ta alla layers bgazei to regret sto forward rˆij = WhTij + b
#5) Ola mazi ftiaxnoun to nevroniko diktuo 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class EmbeddingLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers=1):
        super(EmbeddingLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(input_dim, output_dim))  # Learnable weight matrix
        self.b = nn.Parameter(torch.zeros(output_dim))  # Learnable biases
        self.num_layers = num_layers

    def forward(self, x):
        h = x
        for _ in range(self.num_layers):
            h = torch.matmul(h, self.W)
            h += self.b
        return h

class GATLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=8, concat=True):
        super(GATLayer, self).__init__()
        self.gat = GATConv(input_dim, output_dim, heads=num_heads, concat=concat, dropout=0.6)

    def forward(self, x, edge_index):
        #print(x)
        x = self.gat(x, edge_index)
        return F.elu(x)

class FeedForward(nn.Module):
    def __init__(self, input_size, output_size=512):
        super(FeedForward, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

    
class Output_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Output_layer, self).__init__()
        self.W = nn.Parameter(torch.randn(input_dim, output_dim))  # Learnable weight matrix
        self.b = nn.Parameter(torch.zeros(output_dim))  # Learnable biases
        
    def forward(self, x):
        h0ij = torch.matmul(x, self.W)  # Assuming x has shape (batch_size, num_edges, input_dim)
        h0ij += self.b
        return h0ij
    
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GNNModel, self).__init__()
        self.embedding_layer = EmbeddingLayer(input_dim, hidden_dim)
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim)
        self.gat_layers = nn.ModuleList([GATLayer(hidden_dim, hidden_dim // 8, num_heads=8) for _ in range(num_layers)])
        self.feedforward = FeedForward(hidden_dim, hidden_dim)  # Adjusted output_size
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.output_layer = Output_layer(hidden_dim, output_dim)

    def forward(self, x, edge_index1):
        x1 = self.embedding_layer(x)
        #print("After embedding layer:", x1.shape)
        #print("After embedding layer:", edge_index1)
        for layer in self.gat_layers:
            x2 = layer(x1, edge_index1)
            #print("After GAT layer:", x2.shape)
            x1 = x1 + x2

        combined1 = x1
        bn_output1 = self.batch_norm(combined1)
       # print("After first batch norm:", bn_output1.shape)
        
        # Apply feedforward operation
        ff_output = self.feedforward(bn_output1)
        #print("After feedforward layer:", ff_output.shape)
        
        # Add the original input with the feedforward output
        combined = bn_output1 + ff_output
       # print("After adding feedforward output:", combined.shape)
        
        # Apply batch normalization
        bn_output = self.batch_norm(combined)
        #print("After second batch norm:", bn_output.shape)
        
        x = self.output_layer(bn_output)
        #print("After output layer:", x.shape)
        return x
    
# Define model parameters
dx = 1  # Dimension of edge features
dh = 120  # Dimension of hidden layers
output_dim = 1  # Output dimension
num_layers = 5 # Number of GNN layers

nf = torch.tensor(node_features1, dtype=torch.float32).unsqueeze(1)


#adjacency_matrix = torch.tensor(nx.adjacency_matrix(L).todense(), dtype=torch.float32)


# Instantiate the GNN model
model = GNNModel(dx, dh, output_dim, num_layers)
#print(adjacency_matrix)
# Forward pass

#edge_index1= torch.tensor(edge_index, dtype=torch.long)


#edge_index1 = edge_index1.t().contiguous()
#print(edge_index1)
output = model(nf, edge_index_lg) #adjacency_matrix, 
print("Output shape:", output)




Output shape: tensor([[ 11.7973],
        [-17.3433],
        [-13.6178],
        ...,
        [-22.3886],
        [  1.7854],
        [ 11.8960]], grad_fn=<AddBackward0>)


In [43]:
#print("Output shape:", output)


In [44]:

def make_edge_index_and_regret(output,edge_index):
    output = torch.squeeze(output, 1)
    output1= output.detach().numpy()
    node_index2 = []
    node_index1 = []
    node_index3 = []
    regret_values = []
    
    
    #nodes=[]
    #for node in L.nodes(data=True):
        #nodes.append(node[0])
    for i in range(len(edge_index)):
        node_index3.append(edge_index[i])
        regret_values.append(output1[i])

        
    for i in range(len(edge_index)):
        node_index2.append(edge_index[i][1])
        node_index1.append(edge_index[i][0])
        node_index3.append((node_index2[i], node_index1[i]))
        regret_values.append(output1[i])
        
   
    return node_index3,regret_values

def find_the_value(edges,nodes):
    return nodes.index((edges[0],edges[1]))

node_index3,regret_values = make_edge_index_and_regret(output,edge_index)
#print(node_index3)

In [45]:

import numpy as np

#edw einai mia apli me8odos get() gia na mporoume na xwrisoume apo (1,2) dedomeno se (1) kai (2)
def get_nodes(edge):
    return edge[0], edge[1]

def edge_on_tour(edge, tour):
    tour_edges = tour_to_table(tour)
    return edge in tour_edges or (edge[1], edge[0]) in tour_edges

#edw kanei ar8isi oles tis apostaseis pou epilegonte san diadromh gia na bgalei thn sunoliki apostash
def calculate_distance(path, graph):
    distance = 0
    for i in range(len(path) - 1):
        distance += graph[path[i]][path[i + 1]]
    distance += graph[path[-1]][path[0]]  # Return to starting point
    return distance

#Edw exoume thn prwti me8odo pou einai Greedy Nearest Neighbor sigkekrimena epilegi me8odika tous kombous me thn seira pou
#den exoume episkeftei kai episis exoun to mikrotero regret
def greedy_nearest_neighbor(regret, edges, distances):
    num_cities = distances.shape[0]
    visited = [False] * num_cities
    tour = [0]  # Start from city 0
    current_city = 0
    visited[current_city] = True
    sum_tour_d = 0

    for _ in range(num_cities - 1):
        nearest_city = None
        min_distance = float('inf')
        
        for next_city in range(len(regret)):
            poli1, poli2 = get_nodes(edges[next_city])
            if visited[poli1] == False and visited[poli2] == True and current_city == poli2:
                if regret[next_city] < min_distance:
                    nearest_city = poli1
                    min_distance = regret[next_city]
                    
            elif visited[poli2] == False and visited[poli1] == True and current_city == poli1:
                if regret[next_city] < min_distance:
                    nearest_city = poli2
                    min_distance = regret[next_city]

        #sum_tour.append(min_distance)
        tour.append(nearest_city)
        visited[nearest_city] = True
        current_city = nearest_city
        sum_tour_d = calculate_distance(tour, distances1)

    return tour, sum_tour_d  #sum_tour, 

#Se afti thn me8odo ksana enonoume ta dedomena, apo to na exoume enan pinaka [1,2,3,4] exoume [(1,2), (2,3), (3,4)]
def tour_to_table(tour):
    table = []
    for i in range(len(tour) - 1):
        pair = (tour[i], tour[i + 1])
        table.append(pair)
    return table

def penalize_edges(regret_values, penalties, edges):
    utilities = np.zeros(len(edges))
    
    for i in range(len(edges)):
        edge = edges[i]
        edge_idx = find_the_value(edge, node_index3)
        utilities[i] = regret_values[edge_idx] / (1 + penalties[edge_idx])
    
    max_utility = np.max(utilities)
    max_utility_indices = np.where(utilities == max_utility)[0]
    max_utility_edges = [edges[i] for i in max_utility_indices]

    for edge in max_utility_edges:
        edge_idx = find_the_value(edge, node_index3)
        penalties[edge_idx] += 1

    return penalties, max_utility_edges
#Auth h me8odos pernei dio ari8mous apo ton pinaka (dio kombous apo thn diadromi) kai tous allazei 8esi sth diadromi
def two_opt_swap(path, i, j):
    new_path = path[:i] + path[i:j+1][::-1] + path[j+1:]
    return new_path

#Edw exoume thn me8odo 2opt swap dld pernei dio kombous apo thn diadromi kai tous allazei metaksei tous 
#o algorithoms stamataei kai molis bre8ei kaluteri diadromo (mikroteri apostasi)
def two_opt_local_search(path, graph, best_distance):
    n = len(path)
    best_path = path
    improved = True

    while improved:
        improved = False
        for i in range(1, n - 2):
            for j in range(i + 1, n):
                if j - i == 1:  # No point in reversing if i and j are adjacent
                    continue
                new_path = two_opt_swap(best_path, i, j)
                new_distance = calculate_distance(new_path, graph)
                
                if new_distance < best_distance:
                    best_distance = new_distance
                    best_path = new_path
                    improved = True
                    break
            if improved:
                break

    return best_path, best_distance

#Afti i me8odos allazei 8esi mono se enan kombo sth diadromi gia na beltiosi thn diadromi 
def relocate_algorithm(distance_matrix, old_route, best_distance, max_iterations=1000):
    num_cities = len(old_route)
    current_route = old_route[:]  # Copy the initial route
    current_distance = best_distance
    
    for _ in range(max_iterations):
        city1, city2 = np.random.choice(num_cities, size=2, replace=False)
        
        # Ensure city1 is always before city2 in the tour
        if city1 > city2:
            city1, city2 = city2, city1
            
        new_route = current_route[:city1] + current_route[city1+1:]
        new_route = new_route[:city2] + [current_route[city1]] + new_route[city2:]
        new_distance = calculate_distance(new_route, distance_matrix)

        if new_distance < current_distance:
            current_route = new_route
            current_distance = new_distance

    return current_route, current_distance


def local_search_on_penalized_edges(solution, best_distance, max_utility_edges, node_index3, regret_values, distances):
    improved_tour = solution[:]
    improved_dis = best_distance

    #print("Initial Solution:", improved_tour)
    #print("Penalized Edges:", max_utility_edges)

    # Perform 2-opt swaps focusing on penalized edges
    for edge in max_utility_edges:
        #print("Trying to remove edge:", edge, max_utility_edges)
        for i in range(1, len(improved_tour) - 1):
            for j in range(i + 1, len(improved_tour)):
                if (improved_tour[i], improved_tour[j]) == edge or (improved_tour[j], improved_tour[i]) == edge:
                    new_path = two_opt_swap(improved_tour, i, j)
                    new_distance = calculate_distance(new_path, distances)
                    #print("Old Distance:", best_distance, "New Distance:", new_distance)
                    if new_distance < improved_dis:
                        improved_tour = new_path
                        improved_dis = new_distance
                        print("Removed edge with 2-opt:", edge, "New Distance:", new_distance)

    # Perform relocation focusing on penalized edges
    for edge in max_utility_edges:
        #print("Trying to relocate node in edge:", edge)
        for i in range(1, len(improved_tour) - 1):
            if improved_tour[i] in edge:
                for j in range(len(improved_tour)):
                    if j != i and (improved_tour[j], improved_tour[i]) not in max_utility_edges and (improved_tour[i], improved_tour[j]) not in max_utility_edges:
                        new_path, new_distance = relocate_algorithm(distances, improved_tour, improved_dis, improved_tour[i])
                        #print("Old Distance:", best_distance, "New Distance:", new_distance)
                        if new_distance < improved_dis:
                            improved_tour = new_path
                            improved_dis = new_distance
                            print("Relocated node:", improved_tour[i], "New Distance:", new_distance)

    return improved_tour, improved_dis


def gls(distances, regret_values, node_index3, max_iterations=20):
    initial_solution, initial_distance = greedy_nearest_neighbor(regret_values, node_index3, distances)
    best_solution_t, best_distance_t = two_opt_local_search(initial_solution, distances, initial_distance)
    best_solution, best_distance = relocate_algorithm(distances, best_solution_t, best_distance_t)
    
    #print("greedy_nearest_neighbor:",initial_solution, initial_distance)
    #print("two_opt_local_search:",best_solution_t, best_distance_t)
    print("relocate_algorithm:",best_solution, best_distance)
    
    penalties = [0] * len(regret_values)
    for _ in range(max_iterations):
        edges = tour_to_table(best_solution)
        penalties, max_utility_edges = penalize_edges(regret_values, penalties, edges)
        #
        best_solution, best_distance = local_search_on_penalized_edges(best_solution, best_distance, max_utility_edges, node_index3, regret_values, distances)
    #print(penalties, max_utility_edges)
    #print(node_index3)
    return best_solution, best_distance

# Example usage:
if __name__ == "__main__":
    
    distances1 = ApoKomb1.apostasis_test(node_citys, coords)
    
    

    best_solution, best_distance = gls(distances1, regret_values, node_index3)
    print("Best solution:", best_solution,best_distance)
    #print("Best distance:", best_distance)
    #print(node_index3)

relocate_algorithm: [0, 46, 11, 32, 47, 7, 39, 45, 21, 1, 28, 48, 31, 5, 22, 37, 14, 40, 49, 10, 13, 15, 18, 33, 24, 44, 36, 34, 12, 6, 23, 2, 26, 8, 42, 9, 19, 30, 4, 17, 38, 3, 27, 41, 29, 25, 43, 35, 20, 16] 4078.7614407493247
Relocated node: 23 New Distance: 3965.7577046147694
Relocated node: 23 New Distance: 3928.7381427419796
Relocated node: 23 New Distance: 3926.996124799503
Best solution: [0, 46, 11, 32, 47, 7, 39, 45, 21, 1, 28, 48, 31, 5, 37, 22, 14, 40, 49, 10, 13, 18, 33, 15, 24, 44, 34, 36, 12, 6, 23, 26, 8, 2, 42, 9, 19, 30, 4, 17, 38, 3, 27, 41, 29, 25, 43, 35, 20, 16] 3926.996124799503


In [46]:
print("H beltisti diadromi einai:")
for i in range(len(node_citys)):
    print(node_citys[best_solution[i]])

H beltisti diadromi einai:
Schimatári
Chaïdári
Voúla
Kýthnos
Áno Sýros
Mýkonos
Antíparos
Kímolos
Síkinos
Firá
Pérama
Káto Goúves
Mália
Tzermiádo
Antimácheia
Kos
Kremastí
Megísti
Orestiáda
Néa Výssa
Smínthi
Alistráti
Palaiokómi
Ierissós
Galátista
Thessaloníki
Anatolikó
Skýdra
Kallithéa
Níkaia
Argalastí
Míthymna
Skýros
Mantoúdi
Makrakómi
Áno Kalentíni
Igoumenítsa
Kontokáli
Perivóli
Préveza
Amfilochía
Lixoúri
Pýrgos
Xylókastro
Chiliomódi
Ligourió
Isthmía
Loutráki
Ampelákia
Eleónas


In [37]:
#!pip install --force python_tsp
#!pip install --force nomba

In [47]:
#from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing

#permutation, distance = solve_tsp_dynamic_programming(distances1)
permutation, distance = solve_tsp_local_search(distances1)
#permutation, distance = solve_tsp_dynamic_programming(distances1)
#permutation.append(permutation[0])
#print(distances1)
print(permutation, distance )    
print( best_solution, best_distance)

[0, 2, 23, 12, 36, 34, 44, 24, 15, 33, 18, 13, 10, 49, 26, 8, 47, 7, 37, 22, 14, 40, 5, 31, 48, 28, 1, 21, 39, 45, 32, 11, 46, 20, 43, 29, 25, 27, 3, 38, 17, 4, 30, 19, 9, 6, 42, 41, 35, 16] 3771.4193297726056
[0, 46, 11, 32, 47, 7, 39, 45, 21, 1, 28, 48, 31, 5, 37, 22, 14, 40, 49, 10, 13, 18, 33, 15, 24, 44, 34, 36, 12, 6, 23, 26, 8, 2, 42, 9, 19, 30, 4, 17, 38, 3, 27, 41, 29, 25, 43, 35, 20, 16] 3926.996124799503


In [21]:
#import numpy as np
#import pandas as pd
#import torch
#Sto sigkekrimeno kommati tou kwdika ftiaxnoume random senaria gia kombous gia ekpaideush to TSP
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

def generate_tsp_instances(num_instances, num_nodes):
    instances = []
    for _ in range(num_instances):
        nodes = np.random.rand(num_nodes, 2)
        instances.append(nodes)
    #edw ftiaxnonte ta tixaia senaria kai ta pros8etoume san pinaka se enan megalitero pinaka
   
    return instances


#edw exoume ton ar8im8o pou 8a einai ta senaria p.x 200 senaria apo 5 diaforetikes polis
num_instances = 15

#edw dialegoume ton ari8mo ton poleon analoga me ton ari8mo tou senariou pou exoume
num_nodes = len(coords)

instances = generate_tsp_instances(num_instances, num_nodes)
edge_index_train1 = ApoKomb1.edge_index_training(instances)
distances_train = ApoKomb1.apostasis_training(instances)
distances_train= torch.tensor(distances_train, dtype=torch.float32)


#briskoume tis apostaseis se enan pinaka [n,n] 
distances_train_sq = ApoKomb1.apostasis_test_train(coords,instances)


train_dis =[]
edge_index_train3=[]

edge_index_train2= torch.tensor(edge_index_train1, dtype=torch.long)

for i in range(len(edge_index_train1)):
    train_dis.append(torch.unsqueeze(distances_train[i], 1))
    edge_index_train3.append(edge_index_train2[i].t().contiguous())

    

    

#briskoume ola ta regret gia ola ta train models    
train_model = []
for i in range(num_instances):
    train_model.append(model(train_dis[i], edge_index_train3[i]))
    train_model[i] = torch.squeeze(train_model[i], 1)
    train_model[i]= train_model[i].detach().numpy()
    

    
#print(edge_index_train1[0])

best_dist_train=np.zeros(num_instances)    
best_solution_train= []


for i in range(num_instances):
    best_solution_train,best_dist_train[i] = gls(distances_train_sq[i], train_model[i], edge_index_train1[i])
    #permutation, distance = solve_tsp_dynamic_programming(distances1)
    

    
# Define loss function and optimizer
criterion = nn.MSELoss()  # Assuming L2 loss is mean squared error
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = ExponentialLR(optimizer, gamma=0.99)

# Training loop
num_epochs = 100
batch_size = 32
#print(best_dist_train)

targets= np.zeros(num_instances) 

for i in range(num_instances):
    permutation, targets[i] = solve_tsp_local_search(distances_train_sq[i])

for i in range(num_instances):
    print(targets[i],best_dist_train[i] )
    
    
    
best_dist_train=torch.tensor(best_dist_train, dtype=torch.float32)    
targets=torch.tensor(targets, dtype=torch.float32)
#print(targets)


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i in range(0, len(instances), batch_size):
        batch_instances = instances[i:i + batch_size]
        
        optimizer.zero_grad()
        
        
        #outputs = model(inputs, edge_index1)
        loss = criterion(best_dist_train, targets)
        print(loss)
        #loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(instances):.4f}")

    #print("Output shape:", output)

relocate_algorithm: [0, 21, 11, 4, 2, 30, 36, 29, 41, 35, 8, 6, 9, 13, 37, 32, 7, 28, 40, 46, 49, 34, 20, 27, 19, 17, 26, 14, 33, 23, 15, 5, 10, 43, 48, 18, 16, 42, 22, 3, 1, 39, 47, 44, 31, 38, 24, 45, 25, 12] 643.1223730977567
Relocated node: 49 New Distance: 637.8546487524878
relocate_algorithm: [0, 35, 13, 26, 11, 27, 41, 25, 37, 38, 32, 40, 4, 30, 24, 28, 47, 33, 31, 17, 36, 5, 3, 12, 44, 45, 1, 34, 48, 20, 8, 7, 14, 46, 39, 9, 42, 2, 15, 23, 10, 6, 49, 43, 18, 22, 19, 21, 29, 16] 733.2090982351375
Relocated node: 26 New Distance: 730.0415408521599
relocate_algorithm: [0, 5, 22, 23, 28, 12, 25, 17, 33, 32, 3, 10, 13, 44, 1, 11, 37, 35, 46, 27, 38, 7, 4, 19, 31, 8, 43, 20, 48, 15, 34, 6, 16, 39, 40, 45, 41, 47, 18, 42, 9, 30, 26, 14, 49, 36, 2, 24, 29, 21] 637.8892917239688
Relocated node: 5 New Distance: 637.1700001615925
Relocated node: 5 New Distance: 634.1150792031218
Relocated node: 5 New Distance: 625.8834436328647
Relocated node: 5 New Distance: 624.3784829220099
relocate_al

tensor(1329.6263)
Epoch [75/100], Loss: 88.6418
tensor(1329.6263)
Epoch [76/100], Loss: 88.6418
tensor(1329.6263)
Epoch [77/100], Loss: 88.6418
tensor(1329.6263)
Epoch [78/100], Loss: 88.6418
tensor(1329.6263)
Epoch [79/100], Loss: 88.6418
tensor(1329.6263)
Epoch [80/100], Loss: 88.6418
tensor(1329.6263)
Epoch [81/100], Loss: 88.6418
tensor(1329.6263)
Epoch [82/100], Loss: 88.6418
tensor(1329.6263)
Epoch [83/100], Loss: 88.6418
tensor(1329.6263)
Epoch [84/100], Loss: 88.6418
tensor(1329.6263)
Epoch [85/100], Loss: 88.6418
tensor(1329.6263)
Epoch [86/100], Loss: 88.6418
tensor(1329.6263)
Epoch [87/100], Loss: 88.6418
tensor(1329.6263)
Epoch [88/100], Loss: 88.6418
tensor(1329.6263)
Epoch [89/100], Loss: 88.6418
tensor(1329.6263)
Epoch [90/100], Loss: 88.6418
tensor(1329.6263)
Epoch [91/100], Loss: 88.6418
tensor(1329.6263)
Epoch [92/100], Loss: 88.6418
tensor(1329.6263)
Epoch [93/100], Loss: 88.6418
tensor(1329.6263)
Epoch [94/100], Loss: 88.6418
tensor(1329.6263)
Epoch [95/100], Loss: 

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from python_tsp.exact import solve_tsp_dynamic_programming

# Convert distances and targets to PyTorch tensors
distances_train_sq = torch.tensor(distances_train_sq, dtype=torch.float32)
best_dist_train = torch.tensor(best_dist_train, dtype=torch.float32)
targets = torch.zeros(num_instances, dtype=torch.float32)

for i in range(num_instances):
    permutation, targets[i] = solve_tsp_dynamic_programming(distances_train_sq[i].numpy())

# Move tensors to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_dist_train = best_dist_train.to(device)
targets = targets.to(device)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Assuming L2 loss is mean squared error
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = ExponentialLR(optimizer, gamma=0.99)

# Training loop
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i in range(0, len(instances), batch_size):
        batch_instances = instances[i:i + batch_size]
        
        # Convert batch instances to PyTorch tensors and move to device
        # Assuming batch_instances is a list of numpy arrays
        batch_inputs = [torch.tensor(inst, dtype=torch.float32).to(device) for inst in batch_instances]

        optimizer.zero_grad()
        
        # Perform forward pass (assuming your model handles batch processing)
        batch_outputs = [model(inputs, edge_index1) for inputs in batch_inputs]  # Modify edge_index1 accordingly if it varies per instance
        
        # Calculate loss for each batch output
        loss = 0
        for j in range(len(batch_outputs)):
            loss += criterion(batch_outputs[j], targets[i + j])
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(instances):.4f}")

print("Training complete")

In [ ]:

distances_train1= torch.tensor(distances_train, dtype=torch.float32)

test =[]


edge_index_test2=[]
# Instantiate the GNN model
#model = GNNModel(dx, dh, output_dim, num_layers)
#print(adjacency_matrix)
# Forward pass
#print(edge_index_test1[i].t().contiguous() )

edge_index_test1= torch.tensor(edge_index_test, dtype=torch.long)
for i in range(len(edge_index_test1)):
    test.append(torch.unsqueeze(distances_train1[i], 1))
    #distances_train1[i]=  torch.unsqueeze(distances_train[i], 1)
    edge_index_test2.append(edge_index_test1[i].t().contiguous())
#print(edge_index_test1[1])
#print(edge_index_test2 )
#print(distances_train1)
for i in range(num_instances):
    
    output = model(test[i], edge_index_test2[i]) #adjacency_matrix, 
    #print("Output shape:", output)



In [ ]:
import networkx as nx
import numpy as mp
#from haversine import haversine

#sto sigkekrimeno meros tou kwdika pernoume ta dedomena apo prin kai ta topo8etoume se enan algorithmo gia na 
#broume tis apostaseis apo ka8e kombo, sth synexia ta bazoume ta apotelesmata os (EDGE FEATURES) kai meta
#Ftiaxnoume ena grafima me auta kai sth sunexia to metatrepoume se gramiko grafima gia na exoume ta (EDGE FEATURES) os (NODE FEATURES)

# Example usage:
# Create a sample graph
G = nx.Graph()
for i in range(len(edge_index)):
    G.add_edge(edge_index[i][0], edge_index[i][1], weight=distances[i])

def line_graph(G):
    L = nx.line_graph(G)
    for node in L.nodes:
        edges = G.edges(node)
        if edges:
            edge_features = G.edges[node]['weight']
            L.nodes[node]['e_f'] = edge_features
        else:
            L.nodes[node]['e_f'] = None
    return L

def line_graph_node_features(G):
    node_features = []
    for _, node_data in G.nodes(data=True):
        node_features.append(node_data['e_f'])
    return node_features

L = line_graph(G)
node_features1 = line_graph_node_features(L)

# Create a mapping from original graph edges to line graph nodes
edge_to_index = {edge: idx for idx, edge in enumerate(L.nodes())}

# Prepare edge index for the line graph
edge_index_t = []
for edge in L.edges:
    src, dst = edge
    edge_index_t.append([edge_to_index[src], edge_to_index[dst]])

# Convert edge list to a 2D tensor
edge_index1 = torch.tensor(edge_index_t, dtype=torch.long).t().contiguous()
print(edge_index1)
# Convert node features to tensor
nf = torch.tensor(node_features1, dtype=torch.float32).unsqueeze(1)


#nx.draw(G, with_labels= True)
# Transform the graph into its line graph representation
print('Original Graph: ',G)


nx.draw(L, with_labels= True)
print('Line Graph: ',L)






In [ ]:
import db1
import ApoKomb1    
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
import dgl
from dgl.nn.pytorch import GATConv
from dgl import LineGraph


# Sample city data
citys = db1.basi()

citys1 = citys.sample(5)
lats_c = citys1[["lat"]]
lngs_c = citys1[["lng"]]
node_citys = citys1[["city"]]

location = citys1[["lat", "lng"]].astype(float)

coords = ApoKomb1.sindet(lats_c, lngs_c)
edge_index = ApoKomb1.edge_index(node_citys)
distances = ApoKomb1.apostasis(node_citys, coords)
print(distances)

# Convert edge_index to a NumPy array
edge_index = np.array(edge_index)

# Create a sample graph using DGL
graph = dgl.graph((edge_index[:, 0], edge_index[:, 1]), num_nodes=len(node_citys))
distances =torch.tensor(distances, dtype=torch.float32).unsqueeze(1)

#graph = dgl.from_scipy(distances, eweight_name='w')
graph.edata['w'] = distances
print(graph.edata['w'] )

transform = LineGraph(backtracking=False)

# Convert to line graph
line_graph = transform(graph)

print(line_graph)

# Prepare node features for the line graph



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx

#edw dimiourgiete to nevroniko diktio to opoio exei eksis kommatia 
#1) To EmbeddingLayer to opoio exei os apotelesma sto forward (h0ij = Wxij + b,)
#2) Ena Graph Attection Layer ftMHA(htij , L(G))
#3) Ena Feed Forward Layer  ftFF(h˙t+1ij)
#4) Ena Output Layer to opoio meta apo epeksergasia apo ola ta alla layers bgazei to regret sto forward rˆij = WhTij + b
#5) Ola mazi ftiaxnoun to nevroniko diktuo 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class EmbeddingLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers=1):
        super(EmbeddingLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(input_dim, output_dim))  # Learnable weight matrix
        self.b = nn.Parameter(torch.zeros(output_dim))  # Learnable biases
        self.num_layers = num_layers

    def forward(self, x):
        h = x
        for _ in range(self.num_layers):
            h = torch.matmul(h, self.W)
            h += self.b
        return h

class GATLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=8, concat=True):
        super(GATLayer, self).__init__()
        self.gat = GATConv(input_dim, output_dim, heads=num_heads, concat=concat, dropout=0.6)

    def forward(self, x, edge_index):
        #print(x)
        x = self.gat(x, edge_index)
        return F.elu(x)

class FeedForward(nn.Module):
    def __init__(self, input_size, output_size=512):
        super(FeedForward, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

    
class Output_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Output_layer, self).__init__()
        self.W = nn.Parameter(torch.randn(input_dim, output_dim))  # Learnable weight matrix
        self.b = nn.Parameter(torch.zeros(output_dim))  # Learnable biases
        
    def forward(self, x):
        h0ij = torch.matmul(x, self.W)  # Assuming x has shape (batch_size, num_edges, input_dim)
        h0ij += self.b
        return h0ij
    
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GNNModel, self).__init__()
        self.embedding_layer = EmbeddingLayer(input_dim, hidden_dim)
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim)
        self.gat_layers = nn.ModuleList([GATLayer(hidden_dim, hidden_dim // 8, num_heads=8) for _ in range(num_layers)])
        self.feedforward = FeedForward(hidden_dim, hidden_dim)  # Adjusted output_size
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.output_layer = Output_layer(hidden_dim, output_dim)

    def forward(self, x, edge_index1):
        x1 = self.embedding_layer(x)
        #print("After embedding layer:", x1.shape)
        #print("After embedding layer:", edge_index1)
        for layer in self.gat_layers:
            x2 = layer(x1, edge_index1)
            #print("After GAT layer:", x2.shape)
            x1 = x1 + x2

        combined1 = x1
        bn_output1 = self.batch_norm(combined1)
       # print("After first batch norm:", bn_output1.shape)
        
        # Apply feedforward operation
        ff_output = self.feedforward(bn_output1)
        #print("After feedforward layer:", ff_output.shape)
        
        # Add the original input with the feedforward output
        combined = bn_output1 + ff_output
       # print("After adding feedforward output:", combined.shape)
        
        # Apply batch normalization
        bn_output = self.batch_norm(combined)
        #print("After second batch norm:", bn_output.shape)
        
        x = self.output_layer(bn_output)
        #print("After output layer:", x.shape)
        return x
    
# Define model parameters
dx = 1  # Dimension of edge features
dh = 120  # Dimension of hidden layers
output_dim = 1  # Output dimension
num_layers = 5 # Number of GNN layers


nf= torch.tensor(node_features1, dtype=torch.float32)
#print(nf)
nf= torch.unsqueeze(nf, 1)
#print(nf)


#adjacency_matrix = torch.tensor(nx.adjacency_matrix(L).todense(), dtype=torch.float32)


# Instantiate the GNN model
model = GNNModel(dx, dh, output_dim, num_layers)
#print(adjacency_matrix)
# Forward pass

#edge_index1= torch.tensor(edge_index, dtype=torch.long)


#edge_index1 = edge_index1.t().contiguous()
#print(edge_index1)
output = model(graph.edata['w'] , line_graph) #adjacency_matrix, 
print("Output shape:", output)


